# YOLO11 Traning

## Install requirements

In [ ]:
%pip install -r yolo11/requirements.txt

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import torch
import os

In [ ]:
train_params = {}

## Prepare dataset

Replace these lines with your own credentials. For example:

rf = Roboflow(api_key='YOUR_API_KEY')

project = rf.workspace('YOUR_WORKSPACE').project('YOUR_PROJECT')

version = project.version(YOUR_VERSION_NUMBER)

dataset = version.download('YOUR_DATASET_FORMAT')

In [ ]:
rf = Roboflow(api_key="00000000000000000000")
project = rf.workspace("team").project("project")
version = project.version(0)
dataset = version.download("format:yolo11")

In [ ]:
dataPath = dataset.location + '/data.yaml'
train_params.update({'data': dataPath})

## Training

### Set device

In [ ]:
if not torch.cuda.is_available():
    device = 'cpu'
elif torch.cuda.device_count() == 1:
    device = 'cuda'
else:
    device = list(range(torch.cuda.device_count()))

train_params.update({'device': device})
print("使用的 device:", device)

### Set model

```user_model_path = "path/to/last.pt"```

In [ ]:
user_model_path = ""

In [ ]:
if os.path.exists(user_model_path):
    model_path = user_model_path
    train_params.update({'resume': True})
else:
    print(f"Specified model path '{user_model_path}' does not exist. Using default model 'yolo11n.pt'.")
    model_path = 'yolo11n.pt'
model = YOLO(model_path)

### Set arguments

<details>
<summary>Click to expand the configuration table</summary>

| Argument       | Type           | Default | Description                                                                                                                                                                                  |
|----------------|----------------|---------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| model          | str            | None    | Specifies the model file for training. Accepts a path to either a .pt pretrained model or a .yaml configuration file. Essential for defining the model structure or initializing weights.       |
| data           | str            | None    | Path to the dataset configuration file (e.g., coco8.yaml). This file contains dataset-specific parameters, including paths to training and validation data, class names, and number of classes. |
| epochs         | int            | 100     | Total number of training epochs. Each epoch represents a full pass over the entire dataset. Adjusting this value can affect training duration and model performance.                          |
| time           | float          | None    | Maximum training time in hours. If set, this overrides the epochs argument, allowing training to automatically stop after the specified duration. Useful for time-constrained scenarios.       |
| patience       | int            | 100     | Number of epochs to wait without improvement in validation metrics before early stopping the training. Helps prevent overfitting by stopping when performance plateaus.                         |
| batch          | int            | 16      | Batch size with three modes: set as an integer (e.g., batch=16), auto mode for 60% GPU memory utilization (batch=-1), or auto mode with a custom fraction (batch=0.70).                         |
| imgsz          | int or list    | 640     | Target image size for training. All images are resized to this dimension before feeding into the model. Affects model accuracy and computational complexity.                                   |
| save           | bool           | True    | Enables saving of training checkpoints and final model weights. Useful for resuming training or model deployment.                                                                             |
| save_period    | int            | -1      | Frequency of saving model checkpoints (in epochs). A value of -1 disables it. Useful for saving interim models during long training sessions.                                                 |
| cache          | bool           | False   | Enables caching of dataset images in memory (True/ram), on disk (disk), or disables it (False). Improves training speed but increases memory usage.                                            |
| device         | int, str, list | None    | Specifies the device(s) for training: single GPU (0), multiple GPUs (0,1), CPU (cpu), or MPS for Apple silicon (mps).                                                                          |
| workers        | int            | 8       | Number of worker threads for data loading (per RANK if multi-GPU). Influences data preprocessing speed, especially useful in multi-GPU setups.                                                |
| project        | str            | None    | Name of the project directory where training outputs are saved. Useful for organizing different experiments.                                                                                  |
| name           | str            | None    | Name of the training run, creating a subdirectory under project for logs and outputs.                                                                                                         |
| exist_ok       | bool           | False   | If True, allows overwriting of an existing project/name directory. Useful for iterative experimentation without manually clearing outputs.                                                    |
| pretrained     | bool           | True    | Determines whether to start training from a pretrained model. Can be boolean or string path to specific weights. Enhances training efficiency and performance.                                 |
| optimizer      | str            | 'auto'  | Chooses the optimizer (SGD, Adam, AdamW, etc.) or auto for automatic selection. Affects convergence speed and stability.                                                                       |
| seed           | int            | 0       | Sets a random seed for training, ensuring reproducibility of results across runs with the same configurations.                                                                                |
| deterministic  | bool           | True    | Forces deterministic algorithms, ensuring reproducibility but may affect performance due to restrictions on non-deterministic methods.                                                        |
| single_cls     | bool           | False   | Treats all classes in multi-class datasets as a single class. Useful for binary classification or object presence detection.                                                                   |
| classes        | list[int]      | None    | Specifies class IDs to train on. Useful for filtering out certain classes during training.                                                                                                    |
| rect           | bool           | False   | Enables rectangular training, optimizing batch composition to minimize padding. May improve efficiency but can affect accuracy.                                                               |
| multi_scale    | bool           | False   | Enables multi-scale training, modifying imgsz by up to ±50%. Trains the model for better multi-imgsz performance at inference.                                                                |
| cos_lr         | bool           | False   | Uses a cosine learning rate scheduler. Adjusts the LR following a cosine curve, helping with better convergence.                                                                               |
| close_mosaic   | int            | 10      | Disables mosaic augmentation in the last N epochs to stabilize training. Set 0 to disable this feature.                                                                                       |
| resume         | bool           | False   | Resumes training from the last checkpoint, automatically loading weights, optimizer state, and epoch count.                                                                                   |
| amp            | bool           | True    | Enables Automatic Mixed Precision (AMP) training, reducing memory use and often speeding up training without much impact on accuracy.                                                          |
| fraction       | float          | 1.0     | Fraction of the dataset to use during training, useful for quick experiments or resource-limited conditions.                                                                                  |
| profile        | bool           | False   | Profiles ONNX and TensorRT speeds during training. Useful for deployment optimization.                                                                                                        |
| freeze         | int or list    | None    | Freezes the first N layers or a list of layers by index, reducing trainable parameters for fine-tuning/transfer learning.                                                                      |
| lr0            | float          | 0.01    | Initial learning rate; e.g., SGD=1E-2, Adam=1E-3. Adjust carefully for stable training and best performance.                                                                                  |
| lrf            | float          | 0.01    | Final learning rate as a fraction of lr0, i.e., lr0 × lrf, used with schedulers for LR adjustments over time.                                                                                 |
| momentum       | float          | 0.937   | Momentum factor for SGD or beta1 for Adam, influencing how prior gradients affect the current update.                                                                                         |
| weight_decay   | float          | 0.0005  | L2 regularization term, penalizing large weights. Helps reduce overfitting.                                                                                                                   |
| warmup_epochs  | float          | 3.0     | Number of epochs for learning rate warmup, gradually ramping from a small LR to lr0. Stabilizes early training.                                                                               |
| warmup_momentum| float          | 0.8     | Initial momentum for the warmup phase, transitioning to the normal momentum by the end of warmup_epochs.                                                                                     |
| warmup_bias_lr | float          | 0.1     | Learning rate for bias parameters during warmup, helping stabilize training in initial epochs.                                                                                                |
| box            | float          | 7.5     | Weight of the bounding box regression loss, influencing prediction of object box coordinates.                                                                                               |
| cls            | float          | 0.5     | Weight of the classification loss, affecting how the model balances classification accuracy vs. box accuracy.                                                                                 |
| dfl            | float          | 1.5     | Weight of the distribution focal loss, used in certain YOLO versions for refined classification.                                                                                             |
| pose           | float          | 12.0    | Weight of pose loss for keypoint-based models, emphasizing accuracy of keypoint predictions.                                                                                                 |
| kobj           | float          | 2.0     | Weight of keypoint objectness loss in pose estimation tasks. Balances detection confidence with pose accuracy.                                                                               |
| nbs            | int            | 64      | Nominal batch size for loss normalization.                                                                                                                                                   |
| overlap_mask   | bool           | True    | If True, merges object masks into one when overlapping, with the smaller mask placed over the larger.                                                                                         |
| mask_ratio     | int            | 4       | Downsample ratio for segmentation masks. A larger ratio reduces mask resolution, saving memory.                                                                                              |
| dropout        | float          | 0.0     | Dropout rate in classification tasks. Helps prevent overfitting by randomly disabling some neurons.                                                                                          |
| val            | bool           | True    | Enables validation during training, regularly checking performance on a validation set.                                                                                                      |
| plots          | bool           | False   | Generates and saves plots of training/validation metrics and prediction examples, providing visual insights into model performance.                                                          |

</details>

In [ ]:
train_params.update({
    'epochs': 300,
    'imgsz': 640,
    
})

In [ ]:
# start train
results = model.train(**train_params)